# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
import requests
import pandas as pd

# 1) Carregar dados para os municípios no estado do Rio de Janeiro
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'

# Fazer a requisição para a API
r = requests.get(url)

# Verificar o status da requisição
print(r.status_code)

# Converter a resposta em JSON
data_json = r.json()

# Criar um DataFrame com os dados
df = pd.DataFrame(data_json['items'])

# Exibir as primeiras linhas do DataFrame para conferência
print(df.head())

200
   id_pleito tipo_interessado     interessado  cod_ibge  uf num_pvl  \
0       6981        Município  Rio de Janeiro   3304557  RJ    None   
1       8999        Município        Itatiaia   3302254  RJ    None   
2       6445        Município         Resende   3304201  RJ    None   
3       5275        Município  Rio de Janeiro   3304557  RJ    None   
4       5618        Município     Nova Iguaçu   3303500  RJ    None   

                                              status          num_processo  \
0                                           Deferido  17944.000472/2007-90   
1                     Arquivado por decurso de prazo  17944.000881/2014-15   
2                                           Deferido  17944.000349/2013-17   
3  Encaminhado à PGFN com manifestação técnica fa...  17944.000075/2014-47   
4                                          Arquivado  17944.000156/2010-13   

         data_protocolo                                      tipo_operacao  \
0  2007-04-30T03:00:00

In [5]:
# 1) Verificar os nomes das colunas no DataFrame
print(df.columns)

# 2) Encontrar os três status mais frequentes (substitua pelo nome correto da coluna de status)
print(df.columns)

# 2) Encontrar os três status mais frequentes usando o nome correto da coluna
top_3_status = df['status'].value_counts().head(3)

print("Os três status mais frequentes e suas frequências:")
print(top_3_status)


Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')
Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')
Os três status mais frequentes e suas frequências:
status
Deferido        106
Arquivado        53
Regularizado     26
Name: count, dtype: int64


In [38]:
# 3) Criar uma nova coluna contendo o ano do status
df['data_status'] = df['data_status'].astype(str)  # Converter para string
df['ano_status'] = df['data_status'].str.slice(0, 4)  # Extrair os primeiros 4 caracteres (ano)

# Conferir as primeiras linhas da nova coluna
print(df[['data_status', 'ano_status']].head())

  data_status ano_status
0  25/05/2007       25/0
1  18/02/2015       18/0
2  17/09/2013       17/0
3  29/05/2014       29/0
4  15/12/2010       15/1


In [40]:
# 4) Frequência de cada ano do campo 'ano_status'
frequencia_ano = df['ano_status'].value_counts()

print("Frequência de cada ano:")
print(frequencia_ano)

Frequência de cada ano:
ano_status
13/0    14
15/0    13
28/0    13
29/0    12
26/0    11
25/0    10
06/0    10
12/0    10
30/1    10
22/0     9
27/0     9
11/0     7
10/0     7
18/0     7
21/0     6
20/0     6
02/0     6
03/0     5
09/0     5
23/0     5
23/1     5
17/0     5
10/1     4
21/1     4
31/0     4
01/1     4
04/0     4
28/1     4
30/0     4
14/0     4
01/0     4
19/1     3
17/1     3
22/1     3
24/0     3
04/1     3
07/0     3
06/1     2
05/0     2
18/1     2
08/0     2
13/1     2
19/0     2
02/1     2
05/1     1
11/1     1
09/1     1
26/1     1
08/1     1
12/1     1
14/1     1
29/1     1
31/1     1
07/1     1
27/1     1
15/1     1
16/0     1
24/1     1
20/1     1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [11]:
# Seu código aqui
import requests
import pandas as pd

# Função para obter dados do PVL de acordo com UF e tipo de interessado
def obter_dados_pvl(uf, tipo_interessado):
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['items'])
        return df
    else:
        print(f"Erro na solicitação: {response.status_code}")
        return None

# Consultar o estado de Minas Gerais com status 'Arquivado por decurso de prazo'
df_mg = obter_dados_pvl('MG', 'Estado')

# Filtrar por status 'Arquivado por decurso de prazo'
solicitacoes_arquivadas = df_mg[df_mg['status'] == 'Arquivado por decurso de prazo']
print(f"Solicitações arquivadas em Minas Gerais: {solicitacoes_arquivadas.shape[0]}")

Solicitações arquivadas em Minas Gerais: 1


In [13]:
# Consultar os municípios da Bahia e encontrar o que tem mais solicitações deferidas
df_ba_municipios = obter_dados_pvl('BA', 'Município')

# Filtrar por status 'Deferido'
solicitacoes_deferidas = df_ba_municipios[df_ba_municipios['status'] == 'Deferido']

# Contar o número de solicitações deferidas por município
municipio_mais_deferido = solicitacoes_deferidas['interessado'].value_counts().idxmax()
print(f"Município da Bahia com mais solicitações deferidas: {municipio_mais_deferido}")

Município da Bahia com mais solicitações deferidas: Luís Eduardo Magalhães


In [17]:
# Salvar um arquivo CSV com os dados da Bahia, interessado = 'Estado'
df_ba_estado = obter_dados_pvl('BA', 'Estado')
# Salvar em um arquivo CSV
df_ba_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)
print("Arquivo CSV salvo com os dados do Estado da Bahia.")

Arquivo CSV salvo com os dados do Estado da Bahia.
